In [30]:
import pandas as pd

# Daten camping.ch
## Import

In [81]:
df_camping = pd.read_csv('campingspider/camping.csv', encoding='utf-8')
df_camping.head()

,url,name,locality,website,star category,activities,tourist pitches,open (seasons 1),open (seasons 2),open (seasons 3),...,laundry dryer / tumble dryer,hiking,lake with sandy beach,lake with stony beach,river,train station,bus,VSC membership,TCS membership,SCCV membership
0,https://camping.ch/de/campingplatz/tessin/meri...,Camping Monte San Giorgio,6866 Meride (TI),NaN,4,8,64.0,19.04.2024 − 29.09.2024,NaN,NaN,...,0,1,0,0,1,0,1,1,1,0
1,https://camping.ch/de/campingplatz/wallis/fies...,Eggishorn,3984 Fiesch (VS),NaN,4,6,127.0,01.01.2024 − 31.12.2024,NaN,NaN,...,0,1,0,0,1,1,1,1,0,0
2,https://camping.ch/de/campingplatz/wallis/les-...,La Médettaz,1923 Les Marécottes (VS),NaN,2,5,30.0,01.05.2024 − 30.09.2024,NaN,NaN,...,1,1,0,0,0,1,1,0,0,0
3,https://camping.ch/de/campingplatz/berner-ober...,Gadmen,3863 Gadmen (BE),NaN,2,11,50.0,01.05.2024 − 31.10.2024,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
4,https://camping.ch/de/campingplatz/tessin/mont...,Bosco della Bella,6998 Monteggio (TI),NaN,0,2,NaN,01.01.2024 − 31.12.2024,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0


## Ersetzen von falschen Werten

In [82]:
# Folgende drei Saisondaten waren falsch hinterlegt (mit null Tagen) und wurden deshalb manuell nachgeschaut. Die restlichen Saisondaten wurden nicht nachgeprüft.
corrections_date = {
    'Camping Bellavista': '12.04.2024 - 13.10.2024',
    'Camping Schaffhausen - Freizeitanlage Rheinwiese': '27.03.2024 - 20.10.2024',
    'Nufenenpark': '01.06.2024 - 30.09.2024'
}

# Funktion zur Korrektur der Datumsangaben
def correct_dates(row):
    if row['name'] in corrections_date:
        return corrections_date[row['name']]
    return row['open (seasons 1)']

# Datumsangaben im DataFrame korrigieren
df_camping['open (seasons 1)'] = df_camping.apply(correct_dates, axis=1)

In [83]:
# Quelle: Internetseite der Campingplätze. Es wurden nur die Campingplätze mit Anzahl Touristenplätze < 10 und 'nan' geprüft.
corrections_pitches = {
    'https://camping.ch/de/campingplatz/ostschweiz/weesen/camping-gaesi?l=true': 70,
    'https://camping.ch/de/campingplatz/waadt-genfersee/forel/camping-de-forel?l=true': 43,
    'https://camping.ch/de/campingplatz/ostschweiz/Altst%25C3%25A4tten/allmend_rheintal?l=true': 150,
    'https://camping.ch/de/campingplatz/wallis/ritzingen/camping-brigga?l=true': 127,
    'https://camping.ch/de/campingplatz/ostschweiz/mammern/seewiese?l=true': 95,
    'https://camping.ch/de/campingplatz/zentralschweiz/seelisberg/naturcamping_seelisberg?l=true': 60,
    'https://camping.ch/de/campingplatz/tessin/monteggio/boscodellabella?l=true': 10,
    'https://camping.ch/de/campingplatz/freiburg-neuenburg-jura/le-noirmont/camping-du-creux-des-biches?l=true': 40,
    'https://camping.ch/de/campingplatz/zentralschweiz/sisikon/camping_bucheli?l=true': 50,
    'https://camping.ch/de/campingplatz/wallis/sembrancher/la-prairie?l=true': 48,
    'https://camping.ch/de/campingplatz/berner-mitelland/rueschegg-heubach/camping-rueschegg-heubach?l=true': 20,
    'https://camping.ch/de/campingplatz/zentralschweiz/morgarten/camping_neselen?l=true': 40,
    'https://camping.ch/de/campingplatz/berner-oberland/boenigen/seeruhe?l=true': 88,
    'https://camping.ch/de/campingplatz/ostschweiz/stein-a-rhein/grenzstein?l=true': 64,
    'https://camping.ch/de/campingplatz/berner-oberland/saanen/beim-kappeli?l=true': 27,
    'https://camping.ch/de/campingplatz/freiburg-neuenburg-jura/rebeuvilier/du-raimeux?l=true': 31,
    'https://camping.ch/de/campingplatz/berner-oberland/gsteig-b-gstaad/bergcamping-gsteig?l=true': 16,
    'https://camping.ch/de/campingplatz/berner-oberland/schwenden-im-diemtigtal/camping-eggmatte-diemtigtal?l=true': 16,
    'https://camping.ch/de/campingplatz/freiburg-neuenburg-jura/le-landeron/bellerive-le-landeron?l=true': 50,
}

# Funktion zur Korrektur der Datumsangaben
def correct_dates(row):
    if row['url'] in corrections_pitches:
        return corrections_pitches[row['url']]
    return row['tourist pitches']

# Datumsangaben im DataFrame korrigieren
df_camping['tourist pitches'] = df_camping.apply(correct_dates, axis=1)

## Berechnung prozentuale Öffnungszeiten
Tage Jahr: 365 (Vereinfachung)

In [84]:
from datetime import datetime
# Tage von Öffnung bis Schliessung der Saison
def process_column_value(value):
    # leere Zellen:
    if pd.isna(value) or value == "":
        return 0
    try:
        # wenn ein Text vor dem Datum steht, sollen die Zeichen nach dem Doppelpunkt für das Datum verwendet werden
        if ':' in value:
            middle_value = value[value.find(':') + 2: value.find(':') + 12].strip()
            open_date = datetime.strptime(middle_value, '%d.%m.%Y')
            end_date = datetime.strptime(value[-10:].strip(), '%d.%m.%Y')
            return  (end_date - open_date).days
        end_date = datetime.strptime(value[-10:].strip(), '%d.%m.%Y')
        open_date = datetime.strptime(value[:10].strip(), '%d.%m.%Y')
        return (end_date - open_date).days
    except:
        print(f"Fehler bei der Verarbeitung von Wert {value}")
        return 0

# für alle Saison die Tage berechnen (falls mehrere) und zusammenzählen
def calculate_row(row):
    columns = ['open (seasons 1)', 'open (seasons 2)', 'open (seasons 3)', 'open (seasons 4)', 'open (seasons 5)', 'open (seasons 6)', 'open (seasons 7)']
    total = 0
    for col in columns:
        total += process_column_value(row[col])
    return total

# Neue Spalte mit Tagen erstellen, indem die Berechnung auf jede Zeile angewandt wird
df_camping['open[day]'] = df_camping.apply(calculate_row, axis=1)

In [85]:
# prozentualer Anteil der offenen Tage am Gesamtjahr
df_camping['open[%]'] = df_camping['open[day]']/365

In [86]:
df_camping

,url,name,locality,website,star category,activities,tourist pitches,open (seasons 1),open (seasons 2),open (seasons 3),...,lake with sandy beach,lake with stony beach,river,train station,bus,VSC membership,TCS membership,SCCV membership,open[day],open[%]
0,https://camping.ch/de/campingplatz/tessin/meri...,Camping Monte San Giorgio,6866 Meride (TI),NaN,4,8,64.0,19.04.2024 − 29.09.2024,NaN,NaN,...,0,0,1,0,1,1,1,0,163,0.446575
1,https://camping.ch/de/campingplatz/wallis/fies...,Eggishorn,3984 Fiesch (VS),NaN,4,6,127.0,01.01.2024 − 31.12.2024,NaN,NaN,...,0,0,1,1,1,1,0,0,365,1.000000
2,https://camping.ch/de/campingplatz/wallis/les-...,La Médettaz,1923 Les Marécottes (VS),NaN,2,5,30.0,01.05.2024 − 30.09.2024,NaN,NaN,...,0,0,0,1,1,0,0,0,152,0.416438
3,https://camping.ch/de/campingplatz/berner-ober...,Gadmen,3863 Gadmen (BE),NaN,2,11,50.0,01.05.2024 − 31.10.2024,NaN,NaN,...,0,0,0,0,0,1,0,0,183,0.501370
4,https://camping.ch/de/campingplatz/tessin/mont...,Bosco della Bella,6998 Monteggio (TI),NaN,0,2,10.0,01.01.2024 − 31.12.2024,NaN,NaN,...,0,0,1,0,0,0,0,0,365,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,https://camping.ch/de/campingplatz/graubuenden...,Camping Rania,7432 Zillis (GR),NaN,3,6,35.0,01.01.2024 − 31.12.2024,NaN,NaN,...,0,0,1,0,1,0,0,0,365,1.000000
373,https://camping.ch/de/campingplatz/wallis/sust...,Camping Monument,3952 Susten (VS),www.campingmonument.ch,3,6,170.0,01.05.2024 − 24.09.2024,NaN,NaN,...,0,0,1,1,1,0,0,0,146,0.400000
374,https://camping.ch/de/campingplatz/waadt-genfe...,Bois-du-Fey,1430 Orges (VD),NaN,2,6,12.0,01.03.2024 − 31.10.2024,NaN,NaN,...,0,0,0,1,0,0,0,1,244,0.668493
375,https://camping.ch/de/campingplatz/nordwestsch...,Campingplatz Talhaus,4416 Bubendorf (BL),NaN,0,4,10.0,01.01.2024 − 31.12.2024,NaN,NaN,...,0,0,1,1,0,0,0,0,365,1.000000


## Zusammenfassung der Attribute/Faktoren
z.B. Wassersportanlage ist vorhanden, wenn es entweder einen indoor swimming pool, unheated pool oder bathing facilities gibt. Dazu werden die Werte dieser Spalten zusammengezählt und wenn der Wert null ist, bleibt er null, ansonsten wird eine Eins in eine neue Spalte geschrieben.

In [87]:
# Sport Fields
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['sport field'] + row['golf'] + row['tennis']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['sport_fields_result'] = df_camping.apply(calculate_result, axis=1)

In [88]:
# Watersports Facilities
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['indoor swimming pool'] + row['unheated pool'] + row['bathing facilities']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['watersports_facilities_result'] = df_camping.apply(calculate_result, axis=1)

In [89]:
# Rentals
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['boat rental'] + row['bike rental']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['rentals_sporting_goods_result'] = df_camping.apply(calculate_result, axis=1)

In [90]:
# Restaurants, Bars
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['restaurant'] + row['take away']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['restaurant_bars_result'] = df_camping.apply(calculate_result, axis=1)

In [91]:
# shopping
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['shop with limited range'] + row['kiosk'] + row['shopping centre'] + row['shop with rich range']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['shopping_result'] = df_camping.apply(calculate_result, axis=1)

In [92]:
# washing
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['washing machine'] + row['laundry dryer / tumble dryer']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['washing_result'] = df_camping.apply(calculate_result, axis=1)

In [93]:
# water side
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['lake with sandy beach'] + row['lake with stony beach'] + row['river']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['water_side_result'] = df_camping.apply(calculate_result, axis=1)

In [94]:
# public transport
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['train station'] + row['bus']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['public_transport_result'] = df_camping.apply(calculate_result, axis=1)

In [95]:
# membership
# Summe bilden und wenn nicht null, dann eins
def calculate_result(row):
    total = row['VSC membership'] + row['TCS membership'] + row['SCCV membership']
    return 0 if total == 0 else 1

# Ergebnis in neue Spalte speichern
df_camping['membership_result'] = df_camping.apply(calculate_result, axis=1)

## Aufbau verkürzter Tabelle

In [96]:
df_camping_clean =df_camping[['url', 'name', 'locality', 'website', 'sport_fields_result', 'watersports_facilities_result', 'rentals_sporting_goods_result', 'entertainment', 'playground', 'disco', 'wifi', 'TV lounge', 'tourist pitches', 'easy access for disabled people', 'animals allowed', 'kitchen', 'BBQ area', 'open[%]', 'restaurant_bars_result', 'shopping_result', 'washing_result', 'star category', 'membership_result', 'hiking', 'water_side_result', 'activities', 'public_transport_result' ]]

In [97]:
df_camping_clean

,url,name,locality,website,sport_fields_result,watersports_facilities_result,rentals_sporting_goods_result,entertainment,playground,disco,...,open[%],restaurant_bars_result,shopping_result,washing_result,star category,membership_result,hiking,water_side_result,activities,public_transport_result
0,https://camping.ch/de/campingplatz/tessin/meri...,Camping Monte San Giorgio,6866 Meride (TI),NaN,0,0,1,0,0,0,...,0.446575,0,0,1,4,1,1,1,8,1
1,https://camping.ch/de/campingplatz/wallis/fies...,Eggishorn,3984 Fiesch (VS),NaN,0,1,0,0,1,0,...,1.000000,1,1,1,4,1,1,1,6,1
2,https://camping.ch/de/campingplatz/wallis/les-...,La Médettaz,1923 Les Marécottes (VS),NaN,0,0,1,0,0,0,...,0.416438,1,1,1,2,0,1,0,5,1
3,https://camping.ch/de/campingplatz/berner-ober...,Gadmen,3863 Gadmen (BE),NaN,0,0,0,0,0,0,...,0.501370,0,0,0,2,1,0,0,11,0
4,https://camping.ch/de/campingplatz/tessin/mont...,Bosco della Bella,6998 Monteggio (TI),NaN,0,0,0,0,0,0,...,1.000000,0,0,0,0,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,https://camping.ch/de/campingplatz/graubuenden...,Camping Rania,7432 Zillis (GR),NaN,0,1,0,0,0,0,...,1.000000,1,0,1,3,0,1,1,6,1
373,https://camping.ch/de/campingplatz/wallis/sust...,Camping Monument,3952 Susten (VS),www.campingmonument.ch,0,1,1,0,1,0,...,0.400000,1,1,1,3,0,1,1,6,1
374,https://camping.ch/de/campingplatz/waadt-genfe...,Bois-du-Fey,1430 Orges (VD),NaN,0,0,0,0,1,0,...,0.668493,0,0,1,2,1,1,0,6,1
375,https://camping.ch/de/campingplatz/nordwestsch...,Campingplatz Talhaus,4416 Bubendorf (BL),NaN,0,0,0,0,0,0,...,1.000000,1,0,1,0,0,1,1,4,1


In [98]:
# Bearbeitete Daten als CSV speichern
df_camping_clean.to_csv('camping_clean.csv', encoding='utf-8', index=False)